<a href="https://colab.research.google.com/github/sammainahkinya1404/A-novel-detection-method-based-on-DETR-for-drone-aerial-images/blob/main/Text_Augmentation_nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd

In [ ]:
!pip install torch torchtext pandas nltk


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
# Load your dataset
# df = pd.read_csv('/content/marachiprocessed_CSVfile.csv')

In [ ]:
# df.shape

In [ ]:
def add_noise(sentence, noise_level=0.1):
    """Introduce random noise into the sentence by adding, swapping, or deleting words."""
    words = sentence.split()
    num_noisy_words = max(1, int(len(words) * noise_level))

    for _ in range(num_noisy_words):
        noise_type = random.choice(['add', 'swap', 'delete'])
        if noise_type == 'add':
            word_to_add = random.choice(words)
            position = random.randint(0, len(words))
            words.insert(position, word_to_add)
        elif noise_type == 'swap' and len(words) > 1:
            pos1, pos2 = random.sample(range(len(words)), 2)
            words[pos1], words[pos2] = words[pos2], words[pos1]
        elif noise_type == 'delete' and len(words) > 1:
            words.pop(random.randint(0, len(words) - 1))

    return ' '.join(words)

In [ ]:


# Augment dataset using text noising
augmented_data = []
for index, row in df.iterrows():
    luyia_text = row['Original']
    swahili_text = row['Translation']

    # Add noise to Luyia text
    noisy_luyia = add_noise(luyia_text)
    augmented_data.append((noisy_luyia, swahili_text))

    # Add noise to Swahili text
    noisy_swahili = add_noise(swahili_text)
    augmented_data.append((luyia_text, noisy_swahili))

# Combine original and augmented data
combined_dataset = df.values.tolist() + augmented_data

# Save the augmented dataset to a file
augmented_df = pd.DataFrame(combined_dataset, columns=['Original', 'Translation'])
augmented_df.to_csv('augmented_dataset.csv', index=False)

print("Augmentation complete. The augmented dataset has been saved to 'augmented_dataset.csv'.")


Augmentation complete. The augmented dataset has been saved to 'augmented_dataset.csv'.


In [ ]:
data=pd.read_csv('/content/augmented_dataset.csv')

In [ ]:
data.shape

(7416, 2)

In [ ]:
data.head(-50)

,Original,Translation
0,isafari eyokhutsia khubebusi bange mombasa,safari ya kuenda kwa wazazi wangu mombasa
1,yalio inyanga ndala eya abebusi bange bananga ...,kulikuwa na siku wazazi wangu walinialika nian...
2,olwa inyanga eyo yola ndetayarisia evilai nga ...,hiyo siku ilipowadia nijitayarisha vizuri nika...
3,lwahuola ekhoere obukula omutoka kwetsa nekwit...,tulipofika nilipofaa kuabiri gari gari lilikuj...
4,obulayi olwa khwanza oluchendo khwenukha obula...,tulipoanza safari tulianza salama na tulipofik...
...,...,...
7361,naye nekesa nende mukhana bachaka khuba tsimba...,naye nekesa na rafike yake walianza kuhadithi ...
7362,kata kario bazee vakhare sibakosa ni vabola mb...,hata hivyo wahenga hawakukosea waliposema ukio...
7363,kata kario bazee vakhare sibakosa ni vabola mb...,waliposema hivyo wahenga hawakukosea hata ukio...
7364,kata kario nekesa nende mukhana kama kawaida y...,hata hivyo nekesa alizidi kutembea hadi akakut...


In [ ]:
!pip install -U torchtext


In [ ]:
import pandas as pd
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

# Load your dataset
df = pd.read_csv('/content/augmented_dataset.csv')

# Tokenize and build vocabularies
def tokenize_text(text):
    return text.split()

# Create fields for source (Luyia) and target (Swahili)
SRC = Field(tokenize=tokenize_text, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_text, init_token='<sos>', eos_token='<eos>', lower=True)

# Create a dataset from the dataframe
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, df, src_field, trg_field):
        self.src_field = src_field
        self.trg_field = trg_field
        self.examples = []
        for i, row in df.iterrows():
            src = row['Original']
            trg = row['Translation']
            self.examples.append((src, trg))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        src, trg = self.examples[idx]
        return self.src_field.preprocess(src), self.trg_field.preprocess(trg)

dataset = TranslationDataset(df, SRC, TRG)

# Build vocabularies
SRC.build_vocab(dataset, min_freq=2)
TRG.build_vocab(dataset, min_freq=2)

# Create iterators
train_iterator, valid_iterator = BucketIterator.splits(
    (dataset, dataset), # Just for demonstration, use proper train/valid split
    batch_size=2,
    sort_key=lambda x: len(x[0]),
    sort_within_batch=True,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)


ModuleNotFoundError: No module named 'torchtext.legacy'